## Products with Least Selling Percentage Analysis


### Objective
- Load and explore the ecommerce sales dataset
- Calculate selling percentages for each product
- Filter products with the lowest selling percentages
- Create interactive visualizations to understand underperforming products

In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [18]:
# Create results directory
import os

# Create results directory if it doesn't exist
results_dir = '../results'
os.makedirs(results_dir, exist_ok=True)
print(f"Results directory created/verified: {results_dir}")

Results directory created/verified: ../results


### 2. Explore Dataset

In [19]:
# Load the raw ecommerce sales data
df = pd.read_csv('../data/raw/ecommerce_sales.csv')

print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
print(df.head())

# Basic statistics
print("\nBasic statistics:")
print(df.describe())

Dataset shape: (1000, 18)

Column names:
['product_id', 'product_name', 'category', 'price', 'review_score', 'review_count', 'sales_month_1', 'sales_month_2', 'sales_month_3', 'sales_month_4', 'sales_month_5', 'sales_month_6', 'sales_month_7', 'sales_month_8', 'sales_month_9', 'sales_month_10', 'sales_month_11', 'sales_month_12']

First few rows:
   product_id    product_name        category   price  review_score  \
0           1          Hoodie        Clothing  190.40           1.7   
1           2    Cookware Set  Home & Kitchen  475.60           3.2   
2           3       Train Set            Toys  367.34           4.5   
3           4      Remote Car            Toys  301.34           3.9   
4           5  Self-Help Book           Books   82.23           4.2   

   review_count  sales_month_1  sales_month_2  sales_month_3  sales_month_4  \
0           220            479            449             92            784   
1           903             21            989            861      

### 3. Selling Percentages

We'll calculate selling percentages based on:
- Total sales across all months
- Average monthly sales performance
- Sales efficiency relative to price point

In [20]:
# Identify sales columns
sales_cols = [col for col in df.columns if 'sales_month' in col]
print(f"Sales columns: {sales_cols}")

# Calculate key metrics
df['total_sales'] = df[sales_cols].sum(axis=1)
df['avg_monthly_sales'] = df[sales_cols].mean(axis=1)
df['sales_variability'] = df[sales_cols].std(axis=1)

# Calculate selling percentage based on total sales relative to maximum possible sales
max_total_sales = df['total_sales'].max()
df['selling_percentage'] = (df['total_sales'] / max_total_sales) * 100

# Calculate price-adjusted selling efficiency
# Higher price should ideally lead to higher sales value, so we normalize by price
df['sales_per_dollar'] = df['total_sales'] / df['price']
max_sales_per_dollar = df['sales_per_dollar'].max()
df['price_adjusted_selling_percentage'] = (df['sales_per_dollar'] / max_sales_per_dollar) * 100

print("\nSelling percentage statistics:")
print(df[['selling_percentage', 'price_adjusted_selling_percentage']].describe())

print(f"\nProducts with 0% selling percentage: {(df['selling_percentage'] == 0).sum()}")
print(f"Products with selling percentage < 10%: {(df['selling_percentage'] < 10).sum()}")

Sales columns: ['sales_month_1', 'sales_month_2', 'sales_month_3', 'sales_month_4', 'sales_month_5', 'sales_month_6', 'sales_month_7', 'sales_month_8', 'sales_month_9', 'sales_month_10', 'sales_month_11', 'sales_month_12']

Selling percentage statistics:
       selling_percentage  price_adjusted_selling_percentage
count         1000.000000                        1000.000000
mean            65.784198                           5.015497
std             10.843337                           8.962297
min             32.477325                           0.631945
25%             58.922522                           1.411440
50%             65.479183                           2.127351
75%             72.997487                           4.369036
max            100.000000                         100.000000

Products with 0% selling percentage: 0
Products with selling percentage < 10%: 0


## 4. Filter Products with Least Selling Percentage

We'll identify products with the lowest selling percentages using different criteria:

In [21]:
# Filter products with lowest selling percentages (bottom 20%)
bottom_20_percent = df['selling_percentage'].quantile(0.2)
least_selling_products = df[df['selling_percentage'] <= bottom_20_percent].copy()

print(f"Threshold for bottom 20%: {bottom_20_percent:.2f}%")
print(f"Number of products in bottom 20%: {len(least_selling_products)}")

# Sort by selling percentage (ascending)
least_selling_products = least_selling_products.sort_values('selling_percentage')

print("\nProducts with least selling percentage:")
print(least_selling_products[['product_name', 'category', 'price', 'total_sales', 'selling_percentage', 'price_adjusted_selling_percentage']].head(15))

# Also get the absolute worst performers (bottom 10 products)
worst_performers = df.nsmallest(10, 'selling_percentage')
print(f"\nTop 10 worst performing products:")
print(worst_performers[['product_name', 'category', 'price', 'total_sales', 'selling_percentage']])

Threshold for bottom 20%: 57.25%
Number of products in bottom 20%: 200

Products with least selling percentage:
        product_name        category   price  total_sales  selling_percentage  \
122  Generic Product          Health  162.41         2972           32.477325   
691  Building Blocks            Toys  215.59         3162           34.553601   
785        Biography           Books  454.68         3286           35.908644   
665            Mixer  Home & Kitchen   80.67         3301           36.072560   
135           Kettle  Home & Kitchen  164.99         3391           37.056059   
178         Yoga Mat          Sports  468.68         3443           37.624303   
94           Monitor     Electronics  386.63         3539           38.673369   
814  Generic Product          Health  435.60         3551           38.804502   
543          T-Shirt        Clothing  337.49         3564           38.946563   
624       Remote Car            Toys  311.04         3641           39.788001 

## 5. Create Plotly Visualizations

Let's create interactive visualizations to better understand the products with least selling percentages.

In [25]:
# 1. Bar chart of products with least selling percentage (Bottom 15 products)
top_15_worst = df.nsmallest(15, 'selling_percentage')

fig1 = px.bar(
    top_15_worst,
    x='selling_percentage',
    y='product_name',
    color='category',
    title='Top 15 Products with Least Selling Percentage',
    labels={
        'selling_percentage': 'Selling Percentage (%)',
        'product_name': 'Product Name',
        'category': 'Category'
    },
    orientation='h',
    hover_data=['price', 'total_sales', 'avg_monthly_sales']
)

fig1.update_layout(
    height=600,
    showlegend=True,
    yaxis={'categoryorder': 'total ascending'}
)

# Save the figure
fig1.write_html(os.path.join(results_dir, 'least_selling_products_bar_chart.html'))

# Try to save as PNG (requires kaleido package)
try:
    fig1.write_image(os.path.join(results_dir, 'least_selling_products_bar_chart.png'), width=1200, height=600, scale=2)
    print("Saved bar chart as HTML and PNG to results directory")
except ValueError as e:
    if "kaleido" in str(e):
        print("Saved bar chart as HTML to results directory")
        print("Note: PNG export requires kaleido package. Install with: pip install kaleido")
    else:
        print(f"Error saving PNG: {e}")
        print("Saved bar chart as HTML to results directory")

fig1.show()

Saved bar chart as HTML to results directory
Note: PNG export requires kaleido package. Install with: pip install kaleido


In [26]:
# 2. Scatter plot: Price vs Selling Percentage for bottom 20% products
fig2 = px.scatter(
    least_selling_products,
    x='price',
    y='selling_percentage',
    color='category',
    size='total_sales',
    hover_name='product_name',
    title='Price vs Selling Percentage (Bottom 20% Products)',
    labels={
        'price': 'Price ($)',
        'selling_percentage': 'Selling Percentage (%)',
        'category': 'Category',
        'total_sales': 'Total Sales'
    }
)

fig2.update_layout(height=500)

# Save the figure
fig2.write_html(os.path.join(results_dir, 'price_vs_selling_percentage_scatter.html'))

# Try to save as PNG (requires kaleido package)
try:
    fig2.write_image(os.path.join(results_dir, 'price_vs_selling_percentage_scatter.png'), width=1200, height=500, scale=2)
    print("Saved scatter plot as HTML and PNG to results directory")
except ValueError as e:
    if "kaleido" in str(e):
        print("Saved scatter plot as HTML to results directory")
        print("Note: PNG export requires kaleido package. Install with: pip install kaleido")
    else:
        print(f"Error saving PNG: {e}")
        print("Saved scatter plot as HTML to results directory")

fig2.show()

Saved scatter plot as HTML to results directory
Note: PNG export requires kaleido package. Install with: pip install kaleido
